# Feature Engineering

Merging topic shares, sentiment, and external data (e.g., IPI).

In [2]:
import pandas as pd

def create_features(monthly_topic_shares_by_publisher_path: str, features_path: str):
    # Load data
    topics = pd.read_csv(monthly_topic_shares_by_publisher_path)
    sentiment = pd.read_csv("../data/processed/monthly_sentiment_all.csv")
    ipi = pd.read_csv("../data/raw/INDPRO.csv")

    # Ensure date formats match
    topics['month'] = pd.to_datetime(topics['month'].astype(str))
    sentiment['month'] = pd.to_datetime(sentiment['date'].astype(str))
    ipi['month'] = pd.to_datetime(ipi['observation_date'].astype(str))

    # For topic data
    topics['month'] = pd.to_datetime(topics['month']).dt.to_period('M').dt.to_timestamp()

    # For sentiment data
    sentiment['month'] = pd.to_datetime(sentiment['date']).dt.to_period('M').dt.to_timestamp()

    # Merge on month + publication
    df = topics.merge(sentiment, on=['month', 'publication'], how='left')
    df = df.merge(ipi, on='month', how='left')

    # Drop rows with missing target (INDPRO)
    df = df.dropna(subset=['INDPRO'])

    # One-hot encode publisher
    df = pd.get_dummies(df, columns=['publication'], drop_first=True)

    # print preview of the final DataFrame
    cols = ['month', 'INDPRO', 'sentiment'] + [c for c in df.columns if c not in ('month', 'INDPRO', 'sentiment')]
    display(df[cols].head())

    # save features to csv
    df.to_csv(features_path, index=False)

LDA Features

In [11]:
create_features(
    monthly_topic_shares_by_publisher_path="../data/processed/monthly_topic_shares_by_publisher.csv",
    features_path="../data/processed/features_lda_monthly.csv"
)

,month,INDPRO,sentiment,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,publication_TMZ,publication_TechCrunch,publication_The Hill,publication_The New York Times,publication_The Verge,publication_Vice,publication_Vice News,publication_Vox,publication_Washington Post,publication_Wired
0,2016-01-01,99.4391,-0.332366,0.047303,0.108757,0.065400,0.177487,0.078956,0.089427,0.147791,...,False,False,False,False,False,False,False,False,False,False
1,2016-01-01,99.4391,-0.359760,0.048579,0.034060,0.039285,0.080410,0.048981,0.047445,0.082206,...,False,False,False,False,False,False,False,False,False,False
2,2016-01-01,99.4391,-0.453634,0.076843,0.100812,0.062806,0.020819,0.084967,0.164064,0.143699,...,False,False,False,False,False,False,False,False,False,False
3,2016-01-01,99.4391,-0.449364,0.178754,0.065679,0.069355,0.092416,0.028589,0.146400,0.133294,...,False,False,False,False,False,False,False,False,False,False
4,2016-01-01,99.4391,-0.171123,0.027663,0.115164,0.163197,0.309055,0.035086,0.034965,0.189210,...,False,False,False,False,False,False,False,False,False,False


BERTopic Features

In [3]:
create_features(
    monthly_topic_shares_by_publisher_path="../data/processed/monthly_topic_shares_by_publisher_bertopic.csv",
    features_path="../data/processed/features_bertopic_monthly.csv"
)

,month,INDPRO,sentiment,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,publication_TMZ,publication_TechCrunch,publication_The Hill,publication_The New York Times,publication_The Verge,publication_Vice,publication_Vice News,publication_Vox,publication_Washington Post,publication_Wired
0,2016-01-01,99.4391,-0.332366,0.211538,0.019231,0.076923,0.115385,0.346154,0.057692,0.096154,...,False,False,False,False,False,False,False,False,False,False
1,2016-01-01,99.4391,-0.359760,0.058824,0.050420,0.008403,0.117647,0.655462,0.058824,0.008403,...,False,False,False,False,False,False,False,False,False,False
2,2016-01-01,99.4391,-0.453634,0.055556,0.416667,0.072222,0.011111,0.027778,0.155556,0.166667,...,False,False,False,False,False,False,False,False,False,False
3,2016-01-01,99.4391,-0.449364,0.189427,0.061674,0.039648,0.048458,0.185022,0.171806,0.145374,...,False,False,False,False,False,False,False,False,False,False
4,2016-01-01,99.4391,-0.171123,0.355422,0.006024,0.015060,0.268072,0.039157,0.012048,0.018072,...,False,False,False,False,False,False,False,False,False,False


sLDA Features

In [13]:
create_features(
    monthly_topic_shares_by_publisher_path="../data/processed/monthly_topic_shares_by_publisher_sLDA.csv",
    features_path="../data/processed/features_slda_monthly.csv"
)

,month,INDPRO,sentiment,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,publication_TMZ,publication_TechCrunch,publication_The Hill,publication_The New York Times,publication_The Verge,publication_Vice,publication_Vice News,publication_Vox,publication_Washington Post,publication_Wired
0,2016-01-01,99.4391,-0.332366,0.113924,0.096957,0.073923,0.136457,0.097931,0.152317,0.141147,...,False,False,False,False,False,False,False,False,False,False
1,2016-01-01,99.4391,-0.359760,0.057679,0.044100,0.071563,0.061600,0.053359,0.114329,0.438747,...,False,False,False,False,False,False,False,False,False,False
2,2016-01-01,99.4391,-0.453634,0.135860,0.049119,0.155220,0.033329,0.082012,0.049961,0.070695,...,False,False,False,False,False,False,False,False,False,False
3,2016-01-01,99.4391,-0.449364,0.035915,0.062259,0.288431,0.050607,0.053273,0.057251,0.194292,...,False,False,False,False,False,False,False,False,False,False
4,2016-01-01,99.4391,-0.171123,0.074303,0.018143,0.032679,0.324294,0.211499,0.059117,0.056758,...,False,False,False,False,False,False,False,False,False,False


LLM Features

In [4]:
create_features(
    monthly_topic_shares_by_publisher_path="../data/processed/monthly_topic_shares_by_publisher_tinyllama.csv",
    features_path="../data/processed/features_tinyllm_monthly.csv"
)

,month,INDPRO,sentiment,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,publication_Mashable,publication_New Republic,publication_New Yorker,publication_Politico,publication_Reuters,publication_The Hill,publication_The Verge,publication_Vice News,publication_Washington Post,publication_Wired
0,2016-01-01,99.4391,-0.359760,0.154844,0.185296,0.203359,0.250485,0.215621,0.409550,0.224827,...,False,False,False,False,False,False,False,False,False,False
1,2016-01-01,99.4391,-0.453634,0.308540,0.273117,0.355583,0.249963,0.360973,0.196366,0.292444,...,False,False,False,False,False,False,False,False,False,False
2,2016-01-01,99.4391,-0.449364,0.200046,0.211304,0.226311,0.232623,0.246707,0.253609,0.244901,...,False,False,False,False,False,False,False,False,False,False
3,2016-01-01,99.4391,-0.171123,0.201018,0.241688,0.210722,0.354613,0.192661,0.209108,0.312546,...,False,False,False,False,False,False,False,False,False,False
4,2016-01-01,99.4391,-0.297804,0.202041,0.306991,0.183948,0.308218,0.204174,0.240398,0.319653,...,False,False,False,False,False,False,False,False,False,False
